In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[7],5
commitnumber,1013f91,a22ab94
conc,-,0.0004
dv,-,0.001
klin,-,6.5e-24
molecule,{'co2': 0.0004},co2
ng_adju,-,"[0, 0]"
ng_refs,-,"[1, 2]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.000901                    1 -0.001030
0.000750        2 -0.000905                    2 -0.001025
0.001052        3 -0.000912                    3 -0.001056
0.001476        4 -0.000919                    4 -0.001068
0.002070        5 -0.000926                    5 -0.001080
0.002904        6 -0.000932                    6 -0.001090
0.004074        7 -0.000937                    7 -0.001100
0.005714        8 -0.000942                    8 -0.001110
0.008015        9 -0.000946                    9 -0.001118
0.011243       10 -0.000950                   10 -0.001129
0.015771       11 -0.000952                   11 -0.001146
0.022122       12 -0.000954                   12 -0.001159
0.031031       13 -0.000955                   13 -0.001171
0.043528       14 -0.000955                   14 -0.001181
0.061057       15 -0.000939                   15 -0.001214
0.085645       16 -0.000823                   16 -0.001230
0.120136       17 -0.000472                   17 -0.001114
0.168516       18  0.000237                   18 -0.000771
0.236378       19  0.001491                   19 -0.000082
0.331549       20  0.003545                   20  0.001117
0.465100       21  0.006742                   21  0.003086
0.652400       22  0.010973                   22  0.005919
0.915100       23  0.012540                   23  0.007147
1.283650       24  0.010449                   24  0.005951
1.800600       25  0.007480                   25  0.004115
2.525700       26  0.004446                   26  0.002177
3.542800       27  0.002170                   27  0.000695
4.969550       28  0.000637                   28 -0.000319
6.970850       29 -0.000356                   29 -0.000985
9.778100       30 -0.000965                   30 -0.001395
13.715850      31 -0.001307                   31 -0.001615
19.239350      32 -0.001469                   32 -0.001698
26.987250      33 -0.001510                   33 -0.001674
37.855300      34 -0.001472                   34 -0.001555
53.100050      35 -0.001329                   35 -0.001392
73.887500      36 -0.001089                   36 -0.001194
97.662500      37 -0.000956                   37 -0.001074
121.437500     38 -0.001106                   38 -0.001219
145.212500     39 -0.001437                   39 -0.001512
168.987500     40 -0.001750                   40 -0.001774
192.762500     41 -0.002026                   41 -0.002049
216.537500     42 -0.002255                   42 -0.002343
240.312500     43 -0.002423                   43 -0.002568
264.087500     44 -0.002524                   44 -0.002699
287.862500     45 -0.002551                   45 -0.002748
311.637500     46 -0.002498                   46 -0.002720
335.412500     47 -0.002363                   47 -0.002579
359.187500     48 -0.002140                   48 -0.002365
382.962500     49 -0.001826                   49 -0.002073
406.737500     50 -0.001419                   50 -0.001680
430.512500     51 -0.000919                   51 -0.001183
454.287500     52 -0.000322                   52 -0.000599
478.062500     53  0.000372                   53  0.000086
501.837500     54  0.001164                   54  0.000875
525.612500     55  0.002055                   55  0.001759
549.387500     56  0.003047                   56  0.002741
573.162500     57  0.004139                   57  0.003833
596.937500     58  0.005333                   58  0.005033
620.712500     59  0.006625                   59  0.006343
644.487500     60  0.008008                   60  0.007728
668.262500     61  0.009453                   61  0.009165
692.037500     62  0.010894                   62  0.010617
715.812500     63  0.012232                   63  0.011998
739.587500     64  0.013420                   64  0.013273
763.362500     65  0.014517                   65  0.014483
78

# Fluxes by Layer

CRD                              CLIRAD  \
                        flug          fldg      fnetg       flug   
pressure    level                                                  
0.000000    1     -34.139633  0.000000e+00 -34.139633 -34.176875   
0.000624    2     -34.139634  2.777869e-08 -34.139633 -34.176875   
0.000876    3     -34.139634  3.921126e-08 -34.139634 -34.176875   
0.001229    4     -34.139634  5.596384e-08 -34.139634 -34.176875   
0.001723    5     -34.139634  8.053480e-08 -34.139634 -34.176875   
0.002417    6     -34.139634  1.165588e-07 -34.139634 -34.176875   
0.003391    7     -34.139634  1.693705e-07 -34.139634 -34.176875   
0.004757    8     -34.139634  2.467833e-07 -34.139634 -34.176875   
0.006672    9     -34.139634  3.602339e-07 -34.139634 -34.176876   
0.009359    10    -34.139635  5.264775e-07 -34.139634 -34.176876   
0.013128    11    -34.139635  7.700408e-07 -34.139635 -34.176877   
0.018415    12    -34.139636  1.126839e-06 -34.139635 -34.176878   
0.025830    13    -34.139638  1.649438e-06 -34.139636 -34.176879   
0.036232    14    -34.139640  2.414899e-06 -34.139638 -34.176881   
0.050823    15    -34.139643  3.549252e-06 -34.139639 -34.176884   
0.071291    16    -34.139647  5.474612e-06 -34.139641 -34.176889   
0.100000    17    -34.139654  9.382501e-06 -34.139645 -34.176896   
0.140271    18    -34.139664  1.770880e-05 -34.139646 -34.176907   
0.196760    19    -34.139680  3.525050e-05 -34.139645 -34.176925   
0.275997    20    -34.139703  7.170838e-05 -34.139631 -34.176952   
0.387100    21    -34.139731  1.463670e-04 -34.139584 -34.176989   
0.543100    22    -34.139757  2.971001e-04 -34.139459 -34.177039   
0.761700    23    -34.139761  5.856681e-04 -34.139176 -34.177097   
1.068500    24    -34.139744  1.024146e-03 -34.138720 -34.177171   
1.498800    25    -34.139743  1.555212e-03 -34.138187 -34.177290   
2.102400    26    -34.139791  2.138882e-03 -34.137653 -34.177484   
2.949000    27    -34.139932  2.725695e-03 -34.137207 -34.177788   
4.136600    28    -34.140199  3.297941e-03 -34.136902 -34.178235   
5.802500    29    -34.140629  3.852897e-03 -34.136775 -34.178860   
8.139200    30    -34.141266  4.391580e-03 -34.136874 -34.179707   
11.417000   31    -34.142167  4.918600e-03 -34.137249 -34.180832   
16.014700   32    -34.143402  5.440920e-03 -34.137961 -34.182296   
22.464000   33    -34.145050  5.966122e-03 -34.139083 -34.184175   
31.510500   34    -34.147201  6.499778e-03 -34.140702 -34.186539   
44.200100   35    -34.149952  7.037891e-03 -34.142915 -34.189421   
62.000000   36    -34.153217  7.499125e-03 -34.145717 -34.192829   
85.775000   37    -34.156599  7.816261e-03 -34.148783 -34.196549   
109.550000  38    -34.159513  8.037998e-03 -34.151475 -34.199829   
133.325000  39    -34.162965  8.374278e-03 -34.154591 -34.203614   
157.100000  40    -34.167692  9.053695e-03 -34.158638 -34.208504   
180.875000  41    -34.173786  1.022005e-02 -34.163566 -34.214508   
204.650000  42    -34.181299  1.202612e-02 -34.169273 -34.221805   
228.425000  43    -34.190255  1.463225e-02 -34.175623 -34.230647   
252.200000  44    -34.200648  1.820158e-02 -34.182447 -34.240992   
275.975000  45    -34.212455  2.290106e-02 -34.189554 -34.252714   
299.750000  46    -34.225636  2.889917e-02 -34.196737 -34.265726   
323.525000  47    -34.240138  3.636443e-02 -34.203773 -34.279989   
347.300000  48    -34.255891  4.546443e-02 -34.210427 -34.295267   
371.075000  49    -34.272818  5.636510e-02 -34.216453 -34.311555   
394.850000  50    -34.290822  6.922755e-02 -34.221595 -34.328850   
418.625000  51    -34.309800  8.420775e-02 -34.225593 -34.346999   
442.400000  52    -34.329642  1.014619e-01 -34.228180 -34.365796   
466.175000  53    -34.350230  1.211426e-01 -34.229087 -34.385214   
489.950000  54    -34.371439  1.433986e-01 -34.228041 -34.405192   
513.725000  55    -34.393138  1.683746e-01 -34.224763 -34.425640   
537.500000  56    -34.415187  1.962120e-01 -34.218975 -34.446230   
561.275000  5

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')